# 🍏 Health & Fitness Agent with Bing Grounding 🍎

Welcome to our **Health & Fitness Agent with Bing Grounding** tutorial! In this notebook, we'll demonstrate how to:

1. **Initialize** a project using Azure AI Foundry.
2. **Create an Agent** with the **BingGroundingTool** for web search.
3. **Ask real-world questions** about health and fitness.
4. **Retrieve and display** answers, including Bing query URLs and disclaimers.

### ⚠️ Important Model Support Note ⚠️
>   Bing grounding is currently only supported in certain Azure OpenAI models (e.g. gpt-4o-0513). <br>
    Make sure you specify a supported model and set the "x-ms-enable-preview": "true" header.

## Prerequisites

- Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
- Grounding with Bing connection in Azure AI Foundry, which has to be provisioned from Azure portal. See ["Setup Bing Grounding"](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup) in the documentation for full details.

<img src="./seq-diagrams/bing-connection.jpeg" alt="Bing Grounding Connection" width="600"/>

- A .env file in the parent directory containing:<br>
    ```
    AI_FOUNDRY_PROJECT_ENDPOINT=<your-ai-foundry-project-endpoint><br>
    MODEL_DEPLOYMENT_NAME=<supported-model><br>
    GROUNDING_WITH_BING_CONNECTION_NAME=<the-name-of-your-bing-connection><br>
    ```

## Let's Explore Grounding with Bing!

We'll integrate **Grounding with Bing** search results into our agent so it can gather extra context from the web. We'll store and display the Bing search query link for transparency. 🎉

<img src="./seq-diagrams/4-bing-grounding.png" alt="Bing Grounding Sequence Diagram" width="800"/>

# 🔐 Authentication Setup

Before running the next cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```
az login --use-device-code
```

This will provide you with a device code and URL to authenticate in your browser, which is useful for:

- Remote development environments
- Systems without a default browser
- Corporate environments with strict security policies

After successful authentication, you can proceed with the notebook cells below.

# 1. Initial Setup

We'll load environment variables from .env and initialize our **AIProjectClient** to manage agents.

In [ ]:
#pragma warning disable OPENAI001

#r "nuget: Azure.Identity, 1.18.0-beta.2"
#r "nuget: Azure.AI.Projects, 1.2.0-beta.5"
#r "nuget: dotenv.net"

using System;
using System.Text;
using System.Globalization;
using System.IO;
using System.ClientModel.Primitives;
using System.Reflection;
using Azure.Identity;
using Azure.AI.Projects;
using Azure.AI.Projects.OpenAI;
using OpenAI.Responses;
using OpenAI.Files;
using OpenAI.VectorStores;
using dotenv.net;  

DotEnv.Load(new DotEnvOptions(envFilePaths: new[] { Path.Combine(".","..", ".env") })); 

In [ ]:
var tenantId = Environment.GetEnvironmentVariable("TENANT_ID");
var projectEndpoint = Environment.GetEnvironmentVariable("AI_FOUNDRY_PROJECT_ENDPOINT");
var modelDeployment = Environment.GetEnvironmentVariable("MODEL_DEPLOYMENT_NAME");
var agentName = "health-resource-agent"; 
Console.WriteLine($"🔑 Using Tenant ID: {tenantId}");

AIProjectClient projectClient;

try
{
    var credential = new DefaultAzureCredential(new DefaultAzureCredentialOptions
    {
        TenantId = tenantId
    });

    projectClient = new AIProjectClient(new Uri(projectEndpoint), credential);
    Console.WriteLine("✅ Successfully initialized AIProjectClient");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error initializing AIProjectClient: {ex.Message}");
    throw;
}

# 2. Create Bing-Grounded Agent 🌐

We'll fetch our Bing connection from AI Foundry and use `BingGroundingTool` to let our agent search the web. Then we'll create a new agent with disclaimers about not being a doctor, etc.

Make sure your `MODEL_DEPLOYMENT_NAME` is set to a Bing-supported model (for example, `gpt-4o-0513`) and that you add the header `{"x-ms-enable-preview": "true"}`.

In [ ]:
async Task<AgentVersion> CreateBingGroundedAgentAsync()
{
    try
    {
        string bingConnectionName = Environment.GetEnvironmentVariable("GROUNDING_WITH_BING_CONNECTION_NAME");
        if(string.IsNullOrEmpty(bingConnectionName))
        {
            Console.WriteLine("⚠️ GROUNDING_WITH_BING_CONNECTION_NAME not set in .env - proceeding without Bing grounding");
            Console.WriteLine("💡 This agent will work but won't have web search capabilities");
            PromptAgentDefinition agentWithoutBingDefinition = new(modelDeployment)
            {
                Instructions = @"
                    You are a health and fitness assistant.
                    Always:
                    1. Provide disclaimers that you are not a medical professional.
                    2. Encourage professional consultation.
                    3. Provide brief, helpful answers based on your training data.
                    4. Note that you don't have access to real-time information.
                ",
                Tools = {} // No Bing grounding tool
            };
            var agentWithoutBing = await projectClient.Agents.CreateAgentVersionAsync(
               agentName: "health-agent-no-bing", 
               options: new (agentWithoutBingDefinition));
            Console.WriteLine($"✅ Created Basic Health Agent (no Bing), ID: {agentWithoutBing.Value.Id}");
            return agentWithoutBing.Value;
        }

        string bingConnectionId;
        try
        {
            var bingConnection = await projectClient.Connections.GetConnectionAsync(bingConnectionName);
            bingConnectionId = bingConnection.Value.Id;
            Console.WriteLine($"🔗 Bing Connection ID: {bingConnectionId}");
        }
        catch (Exception ex)
        {
            Console.WriteLine($"❌ Error getting Bing connection '{bingConnectionName}': {ex.Message}");
            Console.WriteLine("💡 Check if the Bing connection exists in your Azure AI Foundry project");
            try
            {
                var connections = projectClient.Connections.GetConnectionsAsync();
                Console.WriteLine("🔍 Available Connections:");
                await foreach (var conn in connections)
                {
                    Console.WriteLine($"    - {conn.Name} (ID: {conn.Id})");
                }
            }
            catch (Exception listEx)
            {
                Console.WriteLine($"❌ Error listing connections: {listEx.Message}");
            }
            return null;
        }

        BingGroundingAgentTool bingGroundingAgentTool = new(new BingGroundingSearchToolOptions(
            searchConfigurations: [new BingGroundingSearchConfiguration(projectConnectionId: bingConnectionId)]
            )
        );

        PromptAgentDefinition agentWithBingDefinition = new(modelDeployment)
        {
            Instructions = @"
                You are a health and fitness assistant with Bing search capabilities.
                Always:
                1. Provide disclaimers that you are not a medical professional.
                2. Encourage professional consultation.
                3. Use Bing for real-time references when appropriate.
                4. Provide brief, helpful answers.
                5. Include relevant sources and citations from your searches.
            ",
            Tools = { bingGroundingAgentTool }
        };
        var agentWithBing = await projectClient.Agents.CreateAgentVersionAsync(
           agentName: "health-bing-agent", 
           options: new (agentWithBingDefinition));
        Console.WriteLine($"🎉 Created Bing-grounded agent, ID: {agentWithBing.Value.Id}");
        return agentWithBing.Value;
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error creating Bing-grounded agent: {ex.Message}");
        return null;
    }
}

var bingAgent = await CreateBingGroundedAgentAsync();

# 3. Starting Threads & Asking Questions 💬

We'll create conversation for each user query, letting the agent search with Bing to find relevant info. We will review the conversation items in the next step

In [ ]:
ProjectConversation conversation = await projectClient.OpenAI.Conversations.CreateProjectConversationAsync();
Console.WriteLine($"💬 Created conversation with ID: {conversation.Id}");

async Task AskBingQuestion(AgentVersion agent, string question)
{
    try
    {
        Console.WriteLine($"📨 Asking question: '{question}'");
        var responsesClient = projectClient.OpenAI.GetProjectResponsesClientForAgent(
                    defaultAgent: bingAgent,
                    defaultConversationId: conversation.Id);
        var responseResult = await responsesClient.CreateResponseAsync(question);
        Console.WriteLine($"🤖 Completion finished, status: {responseResult.Value.Status}");
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error asking question '{question}': {ex.Message}");
    }
}

if(bingAgent != null)
{
    string[] questions = [
        "What are some new HIIT workout trends I should know about?",
        "What's the current WHO recommendation for sugar intake?",
        "Any news on intermittent fasting for weight management?"
    ];
    foreach (var question in questions)
    {
        await AskBingQuestion(bingAgent, question);
    }
}

# 4. Viewing Bing-Grounded Answers & Citations

We'll retrieve the conversation messages, printing both the user queries and the agent's responses. We'll also extract and display the Bing search query URLs used by the agent to ground its answers.

In [ ]:
#pragma warning disable OPENAI001

async Task ViewBingConversationAsync(string conversationId)
{
    Console.WriteLine($"📝 Conversation ID: {conversationId}");

    try
    {
        await foreach (AgentResponseItem item in projectClient.OpenAI.Conversations.GetProjectConversationItemsAsync(
                        conversationId, order:"asc"
                       ))
        {
            var responseResultItem = item.AsResponseResultItem();
            
            if(responseResultItem is MessageResponseItem messageResponseItem)
            {
                var content = messageResponseItem.Content?[0];
                var role = messageResponseItem.Role.ToString();
                var roleEmoji = role == "User" ? "👤" : "🤖";
                Console.WriteLine($"{roleEmoji} {role}: {content?.Text}");
                if(content.OutputTextAnnotations?.Count > 0)
                {
                    foreach(var annotation in content.OutputTextAnnotations)
                    {
                        if(annotation is UriCitationMessageAnnotation)
                        {
                            var uriAnnotation = annotation as UriCitationMessageAnnotation;
                            var uri = uriAnnotation.Uri;
                            Console.WriteLine($"📄 Citation: URI: {uri}");
                        }
                    }
                }
                Console.WriteLine(new string('-', 40));
            }
        }
    }    
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error retrieving conversation items: {ex.Message}");
    }
}

if(conversation != null)
{
    await ViewBingConversationAsync(conversation.Id);
}

# 5. Cleanup & Best Practices

You can optionally delete the agent once you're done. In production, you might keep it around for repeated usage.

### Best Practices
1. **Accuracy** – Bing search results may include disclaimers or partial info. Encourage verification with credible sources.
2. **Bing Query Display** – For compliance with Bing's use and display requirements, show both website URLs (in the agent's response) and Bing search query URLs. If the model includes citations, display them as well.
3. **Limits** – Keep an eye on usage, rate limits, or policy constraints for Bing.
4. **Privacy** – Filter search queries to avoid sending sensitive data.
5. **Evaluations** – Use Azure AI Evaluation for iterative improvement.

In [ ]:
async Task CleanupBingAgent(AgentVersion agent)
{
    if(agent != null)
    {
        try
        {
            await projectClient.Agents.DeleteAgentAsync(agent.Name);
            Console.WriteLine($"🗑️ Deleted agent with ID: {agent.Name}");
        }
        catch (Exception ex)
        {
            Console.WriteLine($"❌ Error deleting agent '{agent.Name}': {ex.Message}");
        }
    }
}

await CleanupBingAgent(bingAgent);

# Congratulations! 🎉

You've successfully completed the Health & Fitness Agent with Bing Grounding tutorial! Here's what was accomplished:

## ✅ What We Built

### 🌐 Fully Functional Bing-Grounded Agent

- Created a health and fitness agent with real Bing grounding capabilities
- Successfully connected to Azure AI Foundry Bing connection
- Agent can now search the web for real-time health and fitness information
- Configured health-focused instructions with appropriate medical disclaimers

### 🔧 Key Features Demonstrated

1. 🔗 **Successful Bing Connection**

- **Connected to actual Bing service**: Retrieved connection from Azure AI Foundry
- **Real web search capabilities**: Agent can now access current information
- **Working end-to-end**: From connection retrieval to agent responses

2. 💬 **Advanced Question Processing**

- **Successfully processed health and fitness queries with real-time data**:
    - HIIT workout trends (with current information)
    - WHO sugar intake recommendations (latest guidelines)
    - Intermittent fasting for weight management (recent research)
- **Live web search**: Agent searches Bing for up-to-date information
- **Source citations**: Agent provides references from web searches

3. 📋 **Enhanced Response Quality**

- Agent provides health advice enhanced with current web information
- **Real-time accuracy**: Responses reflect latest research and guidelines
- **Source transparency**: Web search results and citations included
- **Medical disclaimers**: Appropriate health advice disclaimers maintained

4. 🧹 **Complete Resource Management**

- Properly cleaned up the Bing-grounded agent
- Demonstrated responsible resource management for production use

# 🎯 Key Concepts Mastered
- **Bing Grounding**: Successfully implemented real-time web search capabilities
- **Connection Management**: Proper retrieval and use of Azure AI Foundry connections
- **Tool Configuration**: Correct array format for search configurations
- **Real-time Information**: Agent can now access current web information
- **Source Citation**: Web search results properly integrated into responses
- **Production Readiness**: Robust error handling and resource management

# 🚀 What's Next?

Continue your Azure AI Agent Service journey with these advanced topics:

- [5-agents-aisearch.ipynb](5-agents-aisearch.ipynb) - Integration with Azure AI Search for enterprise knowledge
- [6-agents-az-functions.ipynb](6-agents-az-functions.ipynb) - Agents that can trigger Azure Functions and workflows

# 🌟 Major Achievement

This notebook now demonstrates fully functional Bing grounding with real web search capabilities! The agent can:

- ✅ Search the web in real-time for current health information
- ✅ Provide up-to-date responses based on latest research and guidelines
- ✅ Include source citations from web search results
- ✅ Maintain health disclaimers while leveraging current information

Ready to explore more advanced agent integrations? Let's continue! 🚀

Happy (grounded) agent building! 🌐🤖